In [1]:
import matplotlib.pyplot as plt
import numpy as np
from gym.wrappers import (
    FilterObservation,
    FlattenObservation,
    RecordVideo,
    RescaleAction,
    TimeLimit,
)
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.env_util import unwrap_wrapper

from ea_train import ARESEACheetah


initializing ocelot...


In [2]:
env = ARESEACheetah(
    incoming_mode="random",
    incoming_values=None,
    misalignment_mode="random",
    misalignment_values=None,
    action_mode="direct",
    magnet_init_mode="random",
    magnet_init_values=None,
    reward_mode="differential",
    target_beam_mode="random",
    target_beam_values=None,
    w_mu_x=1.0,
    w_mu_y=1.0,
    w_on_screen=1.0,
    w_sigma_x=1.0,
    w_sigma_y=1.0,
    w_time=1.0,
)
env = RescaleAction(env, -1, 1)

check_env(env)
